###### Introduction to `keras` and `tensorflow`

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.datasets import load_digits, load_sample_images

from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns
    
%load_ext autoreload
%autoreload 2

## Learning Goals

- Describe the concept of backpropagation
- Explain the use of gradient descent in neural networks
- Use `keras` to code up a neural network model

In [ ]:
digits = load_digits()
flat_image = np.array(digits.data[0]).reshape(digits.data[0].shape[0], -1)
eight_by_eight_image = digits.images[0]

In [ ]:
eight_by_eight_image

In [ ]:
plt.imshow(eight_by_eight_image, cmap='Greys');

In [ ]:
digits.target

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(digits.data,
                                                    digits.target,
                                                    random_state=42,
                                                    test_size=0.2)
X_train.shape

In [ ]:
# Z is the input from our collector, the sum of the weights
# multiplied by the features and the bias

def sigmoid(z):
    '''
    Input the sum of our weights times the pixel intensities, plus the bias
    Output a number between 0 and 1.
    
    '''
    return 1/(1+np.e**(-z))

In [ ]:
# Simulation from last time

w_1 = np.random.normal(0, 0.1, (X_train.shape[1], 1))
w_1.shape

In [ ]:
b_1 = 0

z_1 = X_train.dot(w_1) + b_1
z_1

output = sigmoid(z_1)
y_pred = output > 0.5
y_hat = y_pred.astype(int)
y_hat[:5]

## Back propagation

After a certain number of data points have been passed through the model, the weights will be *updated* with an eye toward optimizing our loss function. (Thinking back to biological neurons, this is like revising their activation potentials.) Typically, this is  done  by using some version of gradient descent.

![bprop](images/BackProp_web.png)

### Function Approximation

[Neural networks are much like computational graphs](https://medium.com/tebs-lab/deep-neural-networks-as-computational-graphs-867fcaa56c9).

And computational graphs can be used [to approximate *any* function](http://neuralnetworksanddeeplearning.com/chap4.html).

### Loss Function

If we're thinking of networks, then, as function approximators, of course we'll want to know how good the approximation is. And so once again we have the idea of a [loss function](https://ml-cheatsheet.readthedocs.io/en/latest/loss_functions.html), which is of course what licenses our thinking of networks as models in the first place.

Many loss functions are available. Your choice will depend in part on whether you're doing a regression or classification problem.

Regression:

- mean / median absolute error
- mean / median squared error
- [Huber loss](https://en.wikipedia.org/wiki/Huber_loss)

Classification:

- Crossentropy
- [Hinge loss](https://en.wikipedia.org/wiki/Hinge_loss)
- [Kullback-Leibler divergence](https://www.countbayesie.com/blog/2017/5/9/kullback-leibler-divergence-explained)

The loss function tells us how well our model performed by comparing the predictions to the actual values.

When we train our models with `keras`, we will watch the loss function's progress across epochs.  A decreasing loss function will show us that our model is **improving**.

The loss function is associated with the nature of our output. In logistic regression, our output was binary, so our loss function was the negative loglikelihood, aka **cross-entropy**.

$ \Large -\ loglikelihood = -\frac{1}{m} * \sum\limits_{i=1}^m y_i\log{p_i} + (1-y_i)\log(1-p_i) $
    

In [ ]:
y_train

In [ ]:
y_train = y_train % 2

In [ ]:
y_train

In [ ]:
y_train*np.log(output) + (1-y_train) * np.log(1-output)

In [ ]:
neg_ll = -1/len(y_train)*np.sum(y_train*np.log(output) + (1-y_train) * np.log(1-output))
neg_ll

For continuous variables, the loss function we have relied on is [MSE or MAE](http://rishy.github.io/ml/2015/07/28/l1-vs-l2-loss/).

Good [resource](https://mattmazur.com/2015/03/17/a-step-by-step-backpropagation-example/) on backpropogation with RMSE loss function.

Here is a good summary of different [loss functions]( https://ml-cheatsheet.readthedocs.io/en/latest/loss_functions.html):
   

## Gradient Descent, Epochs, and Batches

We not only use the the loss function to see how our model is improving; we also use it to update our parameters. The gradient of the loss function is calculated in relation to each parameter of our neural net.

For a deep dive into the fitting process, reference Chapter 11 in [Elements of Statistical Learning](https://web.stanford.edu/~hastie/ElemStatLearn/printings/ESLII_print12.pdf)

Gradient descent can be performed in several different ways.  Unlike the `sklearn` implementation of linear regression, which finds the minimum of the loss with a closed form solution, neural networks move down the gradient **incrementally**.

When we fit our neural nets in Keras, we can set the hyperparameter `verbose` equal to 1, and we will see progress through **epochs**. Setting `verbose` to 2 will show just the epoch numbers as they progress.

At the end of each epoch, **all examples** from are training set have passed through the network.

Different types of gradient descent update the parameters at different times.

### Batch Gradient Descent

The gradient is calculated across all values.  We can find the direction of the gradient, and proceed directly towards the minimum.

The weights are updated with regard to the cost at the **end of an epoch** after all training elements have passed through.

### Stochastic Gradient Descent

Updating the weights after all training examples have passed through can be detrimentally slow.  

SGD updates the weights after each training **example**. SGD requires fewer epochs to achieve quality coefficients. This speeds up gradient descent [significantly](https://machinelearningmastery.com/gradient-descent-for-machine-learning/).

### Mini-Batch Gradient Descent

In mini-batch, we pass a batch, calculate the gradient, update the params, then proceed to the next batch. It combines the advantages of batch and stochastic gradient descent: it is faster than SGD since the updates are not made with each point, and more computationally efficient than batch, since we don't have to hold all training examples in memory.

[Good comparison of types of Gradient Descent and batch size](https://machinelearningmastery.com/gentle-introduction-mini-batch-gradient-descent-configure-batch-size/)

## Optimizers

One of the levers we can tweak are the optimizers which control how the weights and biases are updated.

For stochastic gradient descent, the weights are updated with a **constant** learning rate (alpha) after every record.  If we specify a batch size, the constant learning rate is multiplied by the gradient across the batch. 

Other optimizers, such as **Adam** ("Adaptive Moment Estimation"), update the weights in different ways. For Adam,
> A learning rate is maintained for each network weight (parameter) and separately adapted as learning unfolds. See [here](https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/).

![backprop](images/ff-bb.gif)

The graphic above can be a bit frustrating since it moves fast, but follow the progress as so:

Forward propagation with the **blue** tinted arrows computes the output of each layer: i.e. a summation and activation.

Backprop calculates the partial derivative (**green** circles) for each weight (**brown** line) and bias.

Then the optimizer multiplies a **learning rate** ($\eta$) to each partial derivative to calculate a new weight which will be applied to the next batch that passes through.

## Tensorflow and Keras

In [ ]:
from tensorflow import keras

Wait a second, what is that warning? 
`Using TensorFlow backend.`

<img align =left src="images/keras.png"><br>
### Keras is an API

It can be layered on top of many different back-end processing systems.

![kerasback](images/keras_tf_theano.png)

While each of these systems has its own coding methods, `keras` abstracts from that in the streamlined Pythonic manner we are used to seeing in other Python modeling libraries.

Keras development is backed primarily by Google, and the Keras API comes packaged in TensorFlow as tf.keras. Additionally, Microsoft maintains the CNTK Keras backend. Amazon AWS is maintaining the Keras fork with MXNet support. Other contributing companies include NVIDIA, Uber, and Apple (with CoreML).

Theano has been discontinued.  The last release was 2017, but can still be used.

We will use TensorFlow, as it is the most popular. TensorFlow became the most used Keras backend, and  eventually integrated Keras in via its `tf.keras` submodule.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

## Building a Binary Classifier NN

In [ ]:
digits = load_digits()
X = digits.data.astype('float32')
y = digits.target.astype('float32')

We will start with a binary classification, and predict whether the number will be even or odd.

In [ ]:
y_binary = y % 2
y_binary

### Initialize a Linear Stack of Layers

In [ ]:
model = Sequential()

In [ ]:
model

### Add Densely Connected Layers

In [ ]:
model.add(Dense(12, activation='relu', input_dim=64))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

### Compile the Model

The next step is new: After building the model we'll now **compile** it, which is a matter of yoking together the architecture with:
- the optimizer we want to use,
- the [loss function](https://www.analyticsvidhya.com/blog/2021/05/guide-for-loss-function-in-tensorflow/) we want to use, and
- the metrics we want to use.

In [ ]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

### Fit the Model

Now we're ready to **fit** it!

In [ ]:
X.shape

In [ ]:
model.fit(X, y_binary, epochs=50, batch_size=10, verbose=2)

In [ ]:
print(model.predict(X[3].reshape(1,-1)))
print(y[3], y_binary[3])
plt.imshow(X[3].reshape(8,8), cmap='Greys')

In [ ]:
print(model.predict(X[4].reshape(1,-1)))
print(y[4], y_binary[4])
plt.imshow(X[4].reshape(8,8), cmap='Greys')

## Appendix: More on Tensorflow Vs. Keras


### Let's start with tensors

Tensors are multidimensional matrices.

![tensor](images/tensors.png)

### TensorFlow manages the flow of matrix math

That makes neural network processing possible.

![cat](images/cat-tensors.gif)

For our numbers dataset, our tensors from the `sklearn` dataset were originally tensors of the shape 8x8, i.e. 64-bit pictures. Remember, that was with black and white images.

For image processing, we are often dealing with color.

In [ ]:
image = load_sample_images()['images'][0]

imgplot = plt.imshow(image)

In [ ]:
image.shape

What do the dimensions of our image above represent?

Tensors with higher numbers of dimensions have a higher **rank**.

A matrix with rows and columns only, like the black and white numbers, has **rank 2**.

A matrix with a third dimension, like the color pictures above, has **rank 3**.

When we flatten an image by stacking the rows in a column, we are decreasing the rank. 

In [ ]:
flat_image = image.reshape(-1, 1)

flat_image.shape

In [ ]:
427*640*3

## TensorFlow has more levers and buttons, but Keras is more user-friendly

Coding directly in **Tensorflow** allows you to tweak more parameters to optimize performance. The **Keras** wrapper makes the code more accessible for developers prototyping models.

![levers](images/levers.jpeg)

### Keras, an API with an intentional UX

- Deliberately design end-to-end user workflows
- Reduce cognitive load for your users
- Provide helpful feedback to your users

[full article here](https://blog.keras.io/user-experience-design-for-apis.html)<br>
[full list of why to use Keras](https://keras.io/why-use-keras/)

### A few comparisons

While you **can leverage both**, here are a few comparisons.

| Comparison | Keras | Tensorflow|
|------------|-------|-----------|
| **Level of API** | high-level API | High and low-level APIs |
| **Speed** |  can *seem* slower |  is a bit faster |
| **Language architecture** | simple architecture, more readable and concise | straight tensorflow is a bit more complex |
| **Debugging** | less frequent need to debug | difficult to debug |
| **Datasets** | usually used for small datasets | high performance models and large datasets that require fast execution|

This is also a _**non-issue**_ - as you can leverage `tensorflow` commands within `keras` and vice versa. If Keras ever seems slower, it's because the developer's time is more expensive than the GPUs'. Keras is designed with the developer in mind. 

[reference link](https://www.edureka.co/blog/keras-vs-tensorflow-vs-pytorch/)